In [1]:
import pandas as pd
import requests
import json
import datetime as dt
pd.set_option('display.max_columns', None)

In [2]:
weather_api_url = "https://archive-api.open-meteo.com/v1/archive?"

In [3]:
df_results = pd.read_csv("results.csv")
df_races = pd.read_csv("races.csv")
df_drivers = pd.read_csv("drivers.csv")
df_driver_stand = pd.read_csv("driver_standings.csv")
df_constructor = pd.read_csv("constructors.csv")
df_constructor_stand = pd.read_csv("constructor_standings.csv")
df_circuits = pd.read_csv("circuits.csv")
df_pits = pd.read_csv("pit_stops.csv")
df_pits.rename(columns={'time': 'pitTime', 'duration': 'pitDuration'}, inplace=True)

df_races.rename(columns={'name': 'raceName', 'time': 'raceTime'}, inplace=True)
df_constructor_stand.rename(columns={'wins': 'constructorWins', 'position': 'constructorPosition'}, inplace=True)

In [4]:
df_race_n_resuls = pd.merge(df_results, df_races, on=["raceId"])
df_all_about_races = pd.merge(df_race_n_resuls, df_circuits, on=["circuitId"])
df_all_about_drivers = pd.merge(df_driver_stand[['raceId', 'driverId', 'wins']], df_drivers, on=["driverId"])
df_all_about_constructors = pd.merge(df_constructor, df_constructor_stand, on = ["constructorId"])

df_all_about_constructors.rename(columns={'nationality': 'constructorNationality'}, inplace=True)
df_race_n_resuls['year'] = df_race_n_resuls['year'].astype('int')

df_all_about_races = df_all_about_races.drop(columns=["number"])
df_all_about_drivers = df_all_about_drivers.drop(columns=['number'])
#df_all_about_races

In [5]:
df_all_about_drivers.columns

Index(['raceId', 'driverId', 'wins', 'driverRef', 'code', 'forename',
       'surname', 'dob', 'nationality', 'url'],
      dtype='object')

In [6]:
df_all_about_races.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'grid', 'position',
       'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'round', 'circuitId', 'raceName',
       'date', 'raceTime', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date',
       'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time',
       'sprint_date', 'sprint_time', 'circuitRef', 'name', 'location',
       'country', 'lat', 'lng', 'alt', 'url_y'],
      dtype='object')

In [7]:
df_all_about_constructors.columns

Index(['constructorId', 'constructorRef', 'name', 'constructorNationality',
       'url', 'constructorStandingsId', 'raceId', 'points',
       'constructorPosition', 'positionText', 'constructorWins'],
      dtype='object')

In [8]:
df_all_about_constructors.rename(columns={'name': 'constructorName', 'points': 'constructorPoints', 'poisition': 'constructorPosition', 'positionText': 'constructorPositionText'}, inplace=True)

In [9]:
df_races_n_driver = pd.merge(df_all_about_races, df_all_about_drivers, on = ["raceId", "driverId"])
#df_races_n_driver

In [10]:
df_races_n_driver.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'grid', 'position',
       'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'round', 'circuitId', 'raceName',
       'date', 'raceTime', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date',
       'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time',
       'sprint_date', 'sprint_time', 'circuitRef', 'name', 'location',
       'country', 'lat', 'lng', 'alt', 'url_y', 'wins', 'driverRef', 'code',
       'forename', 'surname', 'dob', 'nationality', 'url'],
      dtype='object')

In [11]:
df_race_drive_constructor = pd.merge(df_races_n_driver, df_all_about_constructors, on = ["raceId", "constructorId"], suffixes=['_z', '_w'])

In [12]:
df_race_drive_constructor.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'grid', 'position',
       'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'round', 'circuitId', 'raceName',
       'date', 'raceTime', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date',
       'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time',
       'sprint_date', 'sprint_time', 'circuitRef', 'name', 'location',
       'country', 'lat', 'lng', 'alt', 'url_y', 'wins', 'driverRef', 'code',
       'forename', 'surname', 'dob', 'nationality', 'url_z', 'constructorRef',
       'constructorName', 'constructorNationality', 'url_w',
       'constructorStandingsId', 'constructorPoints', 'constructorPosition',
       'constructorPositionText', 'constructorWins'],
      dtype='object')

In [13]:
#df_pits = df_pits.drop(columns=["milliseconds"])
df_final = pd.merge(df_race_drive_constructor, df_pits, on = ["raceId", "driverId"])
df_final = df_final.drop(columns = ["constructorRef", "constructorNationality","constructorStandingsId", "dob", "driverRef", "circuitRef", 'url_x', 'url_y','url_z', 'url_w'])
df_final = df_final.drop([x for x in df_final if x.endswith('Url')], axis=1)
df_final = df_final[df_final.year >= 2011]
df_final.sort_values(by=['raceId'], inplace=True)
df_final


,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,laps,time,milliseconds_x,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,raceName,date,raceTime,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,name,location,country,lat,lng,alt,wins,code,forename,surname,nationality,constructorName,constructorPoints,constructorPosition,constructorPositionText,constructorWins,stop,lap,pitTime,pitDuration,milliseconds_y
0,20779,841,20,9,1,1,1,1,25.0,58,1:29:30.259,5370259,44,4,1:29.844,212.488,1,2011,1,1,Australian Grand Prix,2011-03-27,06:00:00,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,1,VET,Sebastian,Vettel,German,Red Bull,35.0,1,1,1,1,14,17:25:17,22.603,22603
23,20789,841,153,5,12,11,11,11,0.0,57,\N,\N,41,10,1:30.467,211.025,11,2011,1,1,Australian Grand Prix,2011-03-27,06:00:00,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,0,ALG,Jaime,Alguersuari,Spanish,Toro Rosso,4.0,5,5,0,2,17,17:31:06,24.463,24463
24,20789,841,153,5,12,11,11,11,0.0,57,\N,\N,41,10,1:30.467,211.025,11,2011,1,1,Australian Grand Prix,2011-03-27,06:00:00,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,0,ALG,Jaime,Alguersuari,Spanish,Toro Rosso,4.0,5,5,0,3,35,17:59:45,26.348,26348
25,20787,841,16,10,16,9,9,9,2.0,57,\N,\N,55,13,1:31.526,208.583,11,2011,1,1,Australian Grand Prix,2011-03-27,06:00:00,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,0,SUT,Adrian,Sutil,German,Force India,3.0,6,6,0,1,16,17:29:08,25.978,25978
26,20787,841,16,10,16,9,9,9,2.0,57,\N,\N,55,13,1:31.526,208.583,11,2011,1,1,Australian Grand Prix,2011-03-27,06:00:00,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,0,SUT,Adrian,Sutil,German,Force India,3.0,6,6,0,2,37,18:02:15,23.871,23871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8294,25828,1096,815,9,2,3,3,3,15.0,58,+10.093,5276007,52,4,1:28.972,213.680,1,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,13:00:00,2022-11-18,10:00:00,2022-11-18,13:00:00,2022-11-19,11:00:00,2022-11-19,14:00:00,\N,\N,Yas Marina Circuit,Abu Dhabi,UAE,24.4672,54.6031,3,2,PER,Sergio,Pérez,Mexican,Red Bull,759.0,1,1,17,1,15,17:26:00,21.360,21360
8293,25826,1096,830,9,1,1,1,1,25.0,58,1:27:45.914,5265914,54,6,1:29.392,212.676,1,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,13:00:00,2022-11-18,10:00:00,2022-11-18,13:00:00,2022-11-19,11:00:00,2022-11-19,14:00:00,\N,\N,Yas Marina Circuit,Abu Dhabi,UAE,24.4672,54.6031,3,15,VER,Max,Verstappen,Dutch,Red Bull,759.0,1,1,17,1,20,17:33:31,22.735,22735
8305,25832,1096,839,214,8,7,7,7,6.0,58,+57.240,5323154,45,5,1:29.333,212.817,1,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,13:00:00,2022-11-18,10:00:00,2022-11-18,13:00:00,2022-11-19,11:00:00,2022-11-19,14:00:00,\N,\N,Yas Marina Circuit,Abu Dhabi,UAE,24.4672,54.6031,3,0,OCO,Esteban,Ocon,French,Alpine F1 Team,173.0,4,4,0,1,14,17:24:48,21.895,21895
8321,25841,1096,854,210,12,16,16,16,0.0,57,\N,\N,39,12,1:29.833,211.632,11,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,13:00:00,2022-11-18,10:00:00,2022-11-18,13:00:00,2022-11-19,11:00:00,2022-11-19,14:00:00,\N,\N,Yas Marina Circuit,Abu Dhabi,UAE,24.4672,54.6031,3,0,MSC,Mick,Schumacher,German,Haas F1 Team,37.0,8,8,0,1,14,17:24:59,22.276,22276


In [14]:
df_final.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'grid', 'position',
       'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds_x', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'round', 'circuitId', 'raceName',
       'date', 'raceTime', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time',
       'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date',
       'sprint_time', 'name', 'location', 'country', 'lat', 'lng', 'alt',
       'wins', 'code', 'forename', 'surname', 'nationality', 'constructorName',
       'constructorPoints', 'constructorPosition', 'constructorPositionText',
       'constructorWins', 'stop', 'lap', 'pitTime', 'pitDuration',
       'milliseconds_y'],
      dtype='object')

In [15]:
#initiazlied new empty list
weatherInC = []

df_final.to_csv("F1Stats_noWeather.csv", index = False)


https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2023-05-14&end_date=2023-05-28&hourly=temperature_2m,relativehumidity_2m,precipitation,rain,cloudcover,windspeed_10m

In [ ]:
#df_final = df_final.reset_index()

for i in range(6598):
    latitude = df_final.loc[i, "lat"]
    longitude = df_final.loc[i, "lng"]
    dateStarEnd = df_final.loc[i, "date"]
    raceHour = df_final.loc[i, "raceTimeOfDay"]
    format = '%Y/%m/%d'
    lookUpTime = raceHour[:-5] + "00"
    if len(lookUpTime) == 4:
        lookUpTime = "0" + lookUpTime

    newDate = dt.datetime.strptime(dateStarEnd, format)

    queryHourly = str(newDate.date()) + "T" + lookUpTime

    half_url=weather_api_url+"latitude="+str(latitude)+"&longitude="+str(longitude)+"&start_date="+str(newDate.date())+"&end_date="+str(newDate.date())
    complete_url=half_url+"&hourly=temperature_2m&timezone=auto"

    response = requests.get(complete_url)

    History_Weather = response.json()

    HourlyInfo = History_Weather["hourly"]

    hoursOfDay = HourlyInfo["time"]
    weather_Hourly = HourlyInfo["temperature_2m"]
    
    race_weather = 0

    for j in range(24):
        if hoursOfDay[j] == queryHourly:
            race_weather = weather_Hourly[j]
            
    weatherInC.append(race_weather)    

    

weatherInC


In [ ]:

half_url=weather_api_url+"latitude="+str(latitude)+"&longitude="+str(longitude)+"&start_date="+str(newDate.date())+"&end_date="+str(newDate.date())
complete_url=half_url+"&hourly=temperature_2m&timezone=auto"

In [ ]:
response = requests.get(complete_url)

History_Weather = response.json()

HourlyInfo = History_Weather["hourly"]

hoursOfDay = HourlyInfo["time"]
weather_Hourly = HourlyInfo["temperature_2m"]

for i in (range(len(hoursOfDay))):
    if hoursOfDay[i] == queryHourly:
        race_weather = weather_Hourly[i]

race_weather


13.3